In [45]:
# -*- coding: utf-8 -*-
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn import metrics
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import model_selection
import copy

In [54]:
train = pd.read_csv('../data/train_featureV3.csv',encoding='utf-8')
test = pd.read_csv('../data/test_featureV3.csv',encoding='utf-8')

In [55]:
def getUseColumn(data):
    d = copy.deepcopy(data)
    k = d.var()
    # print k
    print k[(k == np.nan) | (k == 0)].index.values
    col1 = k[(k != np.nan) & (k != 0)].index.values
    return col1

In [56]:
train_x = train.drop(['uid','label'],axis=1)
test_x = test.drop(['uid'],axis=1)
use_cols = getUseColumn(train_x)

dtrain_all = lgb.Dataset(train_x[use_cols].values,label=train.label)
#dtrain_all = lgb.Dataset(train[feat_imp[0:50]['feat'].values].values,label=train.label)


#dtrain = lgb.Dataset(X_train,label=y_train)
#dvalid = lgb.Dataset(X_test,label=y_test)
#dtest = lgb.Dataset(test[feat_imp[0:50]['feat'].values].values)
dtest = lgb.Dataset(test_x[use_cols].values)

[u'voice_opp_len_18' u'voice_opp_len_25'
 u'voice_opp_len_opp_num_unique_18' ...
 u'voice_sms_each_opp_head_count_893' u'voice_sms_each_opp_head_count_955'
 u'voice_sms_each_opp_head_count_991']


In [49]:
train_x.shape

(4999, 6817)

In [50]:
lgb_params =  {
    'boosting_type': 'gbdt',
    'objective': 'binary',
 #  'metric': ('auc', 'f1'),
    #'metric_freq': 100,
    'is_training_metric': False,
    'min_data_in_leaf': 12,
    'num_leaves': 256,
    'learning_rate': 0.05,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.9,
    'verbosity':-1,
    #    'gpu_device_id':2,
#    'device':'gpu',
#     'lambda_l1': 0.1,
#    'skip_drop': 0.95,
#    'max_drop' : 10
#     'lambda_l2': 0.1
    #'num_threads': 18
#     'eta':0.07
}



In [51]:
def evalMetric(preds,dtrain):
    
    label = dtrain.get_label()
    
    
    pre = pd.DataFrame({'preds':preds,'label':label})
    pre= pre.sort_values(by='preds',ascending=False)
    
    auc = metrics.roc_auc_score(pre.label,pre.preds)
    index = int(len(preds) *0.18)
    
    
    pre.preds[0:index] =1
    pre.preds[index:] =0
    
   # pre.preds=pre.preds.map(lambda x: 1 if x>=0.5 else 0)

    f1 = metrics.f1_score(pre.label,pre.preds)
    
   # print 'auc:%f---------f1:%f'%(auc,f1)
    
    
    res = 0.6*auc +0.4*f1
    
    return 'res',res,True
    
    

def auc(preds,dtrain):
    label = dtrain.get_label()
    
    
    pre = pd.DataFrame({'preds':preds,'label':label})
    pre= pre.sort_values(by='preds',ascending=False)
    
    auc = metrics.roc_auc_score(pre.label,pre.preds)
    
    return 'auc',auc,True


def f1(preds,dtrain):
    label = dtrain.get_label()
    
    
    pre = pd.DataFrame({'preds':preds,'label':label})
    pre= pre.sort_values(by='preds',ascending=False)
    
    index = int(len(preds) *0.18)
    
    
    pre.preds[0:index] =1
    pre.preds[index:] =0
    

    f1 = metrics.f1_score(pre.label,pre.preds)
    
    
    return 'f1',f1,True
    
    
    
    

In [52]:
res_cv = lgb.cv(lgb_params,dtrain_all,feval=evalMetric,early_stopping_rounds=100,verbose_eval=10,num_boost_round=10000,nfold=3,metrics=['evalMetric','auc'],seed=1000)
print 'boost_round:%s,res:%s,auc:%s'%(len(res_cv['res-mean']),res_cv['res-mean'][-1],res_cv['auc-mean'][-1])

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


[10]	cv_agg's res: 0.839911 + 0.00376391	cv_agg's auc: 0.928215 + 0.00378463
[20]	cv_agg's res: 0.847893 + 0.00715504	cv_agg's auc: 0.932618 + 0.00675313
[30]	cv_agg's res: 0.852002 + 0.00669625	cv_agg's auc: 0.935756 + 0.00522615
[40]	cv_agg's res: 0.854175 + 0.00600323	cv_agg's auc: 0.937155 + 0.00419292
[50]	cv_agg's res: 0.858107 + 0.00557947	cv_agg's auc: 0.94 + 0.00339179
[60]	cv_agg's res: 0.859881 + 0.00551541	cv_agg's auc: 0.942957 + 0.00329419
[70]	cv_agg's res: 0.862413 + 0.00329366	cv_agg's auc: 0.944953 + 0.00267238
[80]	cv_agg's res: 0.864363 + 0.00593388	cv_agg's auc: 0.945976 + 0.00247281
[90]	cv_agg's res: 0.86661 + 0.00557191	cv_agg's auc: 0.946755 + 0.00269796
[100]	cv_agg's res: 0.866009 + 0.00603467	cv_agg's auc: 0.947237 + 0.00269985
[110]	cv_agg's res: 0.866101 + 0.00579519	cv_agg's auc: 0.94739 + 0.00232926
[120]	cv_agg's res: 0.866436 + 0.00629509	cv_agg's auc: 0.947948 + 0.00258401
[130]	cv_agg's res: 0.865773 + 0.00580196	cv_agg's auc: 0.948326 + 0.00219183
[

In [ ]:
model =lgb.train(lgb_params,dtrain_all,feval=evalMetric,verbose_eval=5,num_boost_round=int(len(res_cv['res-mean'])*1.5))

In [ ]:
pred=model.predict(test.drop(['uid'],axis=1).values)

In [ ]:
test.drop(['uid'],axis=1).values

In [ ]:
res =pd.DataFrame({'uid':test.uid,'label':pred})


In [ ]:
res.to_csv('../result/b/lgb/lgb-%0.4f-%0.4f-prob-nowa.csv'%(res_cv['res-mean'][-1],res_cv['auc-mean'][-1]),index=False,header=False,sep=',',columns=['uid','label'])

In [ ]:
res=res.sort_values(by='label',ascending=False)
index = int(len(res)*0.18)

res.label[0:index] =int(1)
res.label[index:] =int(0)
res.label = res.label.map(lambda x: int(x))

In [ ]:
res.to_csv('../result/b/lgb/lgb-%0.4f-%0.4f-nowa.csv'%(res_cv['res-mean'][-1],res_cv['auc-mean'][-1]),index=False,header=False,sep=',',columns=['uid','label'])